In [2]:
from typing import TypedDict, List, Annotated
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
graph = StateGraph(Job_Agent)
#NODE

graph.add_node('extraction', extraction)
graph.add_node('preprocess', preprocess)
#major part
graph.add_node('missing', missing)
graph.add_node('assestment', assesment)
graph.add_node('voice', voice)
graph.add_node('interview', interview)
#resources
graph.add_node('dsa', dsa)
graph.add_node('job_sites', job_sites)
graph.add_node('courses', courses)

#summay and feedback
graph.add_node('feedback', feedback)

In [ ]:
#EDGE
graph.add_edge(START, 'EOFError')